In [0]:
# ============================================
# CELL 0: ENVIRONMENT VALIDATION
# ============================================

print("=" * 80)
print("🔧 STEP 1: VALIDATING DATABRICKS ENVIRONMENT")
print("=" * 80)

# Check Java version
java_version = spark.sparkContext._jvm.java.lang.System.getProperty("java.version")
print(f"\n✅ Java Version: {java_version}")

# Check Spark version
spark_version = spark.version
print(f"✅ Spark Version: {spark_version}")

# Check Python version
import sys
python_version = sys.version.split()[0]
print(f"✅ Python Version: {python_version}")

# Verify Spark is working
try:
    test_df = spark.range(1, 3)
    test_count = test_df.count()
    print(f"✅ Spark Context: Active (test count = {test_count})")
except Exception as e:
    print(f"❌ Spark Context Error: {e}")
    
print("\n" + "=" * 80)
print("✅ ENVIRONMENT VALIDATION COMPLETE")
print("=" * 80)

In [0]:
# ============================================
# CELL 1: SNOWFLAKE POLARIS CONFIGURATION
# ============================================

print("=" * 80)
print("🔧 STEP 2: CONFIGURING SNOWFLAKE POLARIS CATALOG CONNECTION")
print("=" * 80)

# Snowflake Polaris credentials - UPDATE WITH YOUR VALUES
ACCOUNTADMIN_PAT = "YOUR_SNOWFLAKE_PAT_TOKEN_HERE"  # Get from: Snowflake → Profile → Security → Tokens
SNOWFLAKE_ACCOUNT = "YOUR-SNOWFLAKE-ACCOUNT"  # Format: ORGNAME-ACCOUNTNAME (no underscores!)
DATABASE_NAME = "DEMO_TESTDB"
horizon_role = "session:role:ACCOUNTADMIN"

# Construct Polaris API endpoint
account_uri = f"https://{SNOWFLAKE_ACCOUNT}.snowflakecomputing.com/polaris/api/catalog"

print(f"\n📍 Configuration Details:")
print(f"   Account: {SNOWFLAKE_ACCOUNT}")
print(f"   Database: {DATABASE_NAME}")
print(f"   Polaris URI: {account_uri}")
print(f"   Role: ACCOUNTADMIN")

# Configure Spark to use Snowflake Polaris as Iceberg catalog
print(f"\n⚙️  Configuring Spark catalog 'hz' for Polaris...")

try:
    # Clear any existing catalog configuration first
    try:
        spark.conf.unset("spark.sql.catalog.hz")
        spark.conf.unset("spark.sql.catalog.hz.type")
    except:
        pass  # Ignore if not set
    
    # Core Polaris/Iceberg configuration
    # Use catalog-impl approach (not type) to avoid conflicts
    spark.conf.set("spark.sql.catalog.hz", "org.apache.iceberg.spark.SparkCatalog")
    spark.conf.set("spark.sql.catalog.hz.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
    spark.conf.set("spark.sql.catalog.hz.uri", account_uri)
    spark.conf.set("spark.sql.catalog.hz.warehouse", DATABASE_NAME)
    spark.conf.set("spark.sql.catalog.hz.credential", ACCOUNTADMIN_PAT)
    spark.conf.set("spark.sql.catalog.hz.scope", horizon_role)
    spark.conf.set("spark.sql.catalog.hz.header.X-Snowflake-Authorization-Token-Type", "PROGRAMMATIC_ACCESS_TOKEN")
    spark.conf.set("spark.sql.iceberg.vectorization.enabled", "false")
    
    print("✅ Spark catalog configuration set successfully")
    
except Exception as e:
    print(f"❌ Configuration Error: {e}")
    raise

# Verify configuration was applied
print(f"\n🔍 Verifying configuration...")
catalog_uri = spark.conf.get("spark.sql.catalog.hz.uri")
catalog_warehouse = spark.conf.get("spark.sql.catalog.hz.warehouse")
print(f"   ✅ Catalog URI: {catalog_uri}")
print(f"   ✅ Warehouse: {catalog_warehouse}")

print("\n" + "=" * 80)
print("✅ POLARIS CONFIGURATION COMPLETE")
print("=" * 80)



In [0]:
# ============================================
# CELL 2: TEST CONNECTION & DISCOVERY
# ============================================

print("=" * 80)
print("🔧 STEP 3: TESTING POLARIS CONNECTION & DISCOVERING SCHEMAS")
print("=" * 80)

# Test 1: List namespaces (schemas)
print("\n📋 Test 1: Listing namespaces in Polaris catalog...")
try:
    namespaces_df = spark.sql("SHOW NAMESPACES IN hz")
    namespace_count = namespaces_df.count()
    
    print(f"✅ Successfully connected to Polaris catalog!")
    print(f"✅ Found {namespace_count} namespace(s):\n")
    namespaces_df.show(truncate=False)
    
    # Store namespaces for validation
    # Note: The column might be 'databaseName' or 'namespace' depending on Iceberg version
    try:
        namespaces = [row.databaseName for row in namespaces_df.collect()]
    except:
        namespaces = [row.namespace for row in namespaces_df.collect()]
    
    # Verify PUBLIC schema exists
    if "PUBLIC" in namespaces:
        print("✅ PUBLIC schema found (expected)")
    else:
        print("⚠️  PUBLIC schema not found - check Snowflake setup")
        
except Exception as e:
    print(f"❌ CONNECTION FAILED: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Verify Polaris catalog is enabled in Snowflake")
    print("   2. Check your PAT token is valid")
    print("   3. Ensure database DEMO_TESTDB exists")
    print("   4. Verify Iceberg libraries are installed on cluster")
    raise

# Test 2: List tables in PUBLIC schema
print("\n" + "=" * 80)
print("📋 Test 2: Listing tables in PUBLIC schema...")
try:
    tables_df = spark.sql("SHOW TABLES IN hz.PUBLIC")
    table_count = tables_df.count()
    
    print(f"✅ Found {table_count} table(s) in PUBLIC schema:\n")
    tables_df.show(truncate=False)
    
    # Store table names for validation
    # Get the first row to check available columns
    if table_count > 0:
        first_row = tables_df.first()
        # Try different possible column names
        try:
            tables = [row.tableName for row in tables_df.collect()]
        except:
            try:
                tables = [row.name for row in tables_df.collect()]
            except:
                # Just get all column values from first column
                tables = [row[0] for row in tables_df.collect()]
        
        # Verify USER_INFO table exists
        tables_upper = [t.upper() if t else "" for t in tables]
        if "USER_INFO" in tables_upper:
            print("✅ USER_INFO table found (target table confirmed)")
        else:
            print("⚠️  USER_INFO table not found - check Snowflake table creation")
            print(f"   Available tables: {tables}")
    else:
        print("⚠️  No tables found in PUBLIC schema")
        
except Exception as e:
    print(f"❌ DISCOVERY FAILED: {e}")
    raise

print("\n" + "=" * 80)
print("✅ CONNECTION TEST & DISCOVERY COMPLETE")
print("=" * 80)         

In [0]:
# ============================================
# CELL 3: QUERY USER_INFO TABLE
# ============================================

print("=" * 80)
print("🔧 STEP 4: READING USER_INFO TABLE FROM SNOWFLAKE")
print("=" * 80)

# Query the USER_INFO table
print("\n👥 Querying USER_INFO table via Polaris catalog...")

user_info_df = spark.sql("SELECT * FROM hz.PUBLIC.USER_INFO")

print("\n📊 USER_INFO Table:")
user_info_df.show(truncate=False)

# Show row count
row_count = user_info_df.count()
print(f"\n✅ Total rows: {row_count}")

print("\n" + "=" * 80)
print("✅ QUERY COMPLETE")
print("=" * 80)

In [ ]:
# ============================================
# CELL 4: FINAL VALIDATION & DEMO SUMMARY
# ============================================

print("=" * 80)
print("🎯 FINAL VALIDATION & DEMO SUMMARY")
print("=" * 80)

# Summary of what was accomplished
print("\n✅ DEMO COMPLETED SUCCESSFULLY!\n")

print("📋 What We Accomplished:")
print("   ✅ Step 1: Validated Databricks environment")
print("   ✅ Step 2: Configured Snowflake Polaris catalog connection")
print("   ✅ Step 3: Tested connection and discovered schemas/tables")
print("   ✅ Step 4: Read and validated USER_INFO table")

print("\n🏗️  Architecture Demonstrated:")
print("   ┌─────────────────────────────────────────┐")
print("   │         SNOWFLAKE                        │")
print("   │  Database: DEMO_TESTDB                   │")
print("   │  Table: USER_INFO (Iceberg format)       │")
print("   │  Polaris Catalog (REST API)              │")
print("   └──────────────┬──────────────────────────┘")
print("                  │")
print("                  │ REST API over HTTPS")
print("                  │ (Iceberg protocol)")
print("                  │")
print("   ┌──────────────▼──────────────────────────┐")
print("   │         DATABRICKS                       │")
print("   │  Spark + Iceberg libraries               │")
print("   │  Querying remote Iceberg tables          │")
print("   │  No data duplication!                    │")
print("   └─────────────────────────────────────────┘")

print("\n🔑 Key Capabilities Shown:")
print("   • Data Federation: Query Snowflake data from Databricks")
print("   • Open Standards: Using Apache Iceberg format")
print("   • No ETL: Direct access without copying data")
print("   • Polaris Catalog: Snowflake's REST-based catalog service")

print("\n📊 Table Statistics:")
try:
    final_count = spark.sql("SELECT COUNT(*) as count FROM hz.PUBLIC.USER_INFO").collect()[0]['count']
    print(f"   Total rows in USER_INFO: {final_count}")
    
    final_sample = spark.sql("SELECT * FROM hz.PUBLIC.USER_INFO LIMIT 3")
    print("\n   Sample data:")
    final_sample.show(truncate=False)
    
except Exception as e:
    print(f"   Could not fetch final stats: {e}")

print("\n🚀 Next Steps - Phase 2:")
print("   1. Add more tables (CUSTOMERS, ORDERS, PRODUCTS)")
print("   2. Set up Snowflake Cortex Analyst (semantic model)")
print("   3. Create Cortex Agent for natural language queries")
print("   4. Build demo presentation and talking points")

print("\n" + "=" * 80)
print("✅ DEMO READY FOR PRESENTATION!")
print("=" * 80)

# Create a flag to indicate success
print("\n💾 Saving connection state for future use...")
spark.conf.set("demo.polaris.connection.validated", "true")


In [ ]:
# ============================================
# CELL 5: TROUBLESHOOTING & DIAGNOSTICS (Optional)
# ============================================
# Run this cell only if you encounter errors in previous cells

print("=" * 80)
print("🔧 TROUBLESHOOTING & DIAGNOSTICS")
print("=" * 80)

print("\n🔍 Diagnostic Information:\n")

# Check 1: Verify Iceberg library is installed
print("1️⃣ Checking for Iceberg libraries...")
try:
    iceberg_class = spark.sparkContext._jvm.org.apache.iceberg.spark.SparkCatalog
    print("   ✅ Iceberg libraries are installed")
except Exception as e:
    print("   ❌ Iceberg libraries NOT found!")
    print("   💡 Install: org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.4.3")
    print(f"   Error: {e}")

# Check 2: Verify catalog configuration
print("\n2️⃣ Checking catalog configuration...")
try:
    catalog_impl = spark.conf.get("spark.sql.catalog.hz.catalog-impl")
    catalog_uri = spark.conf.get("spark.sql.catalog.hz.uri")
    print(f"   ✅ Catalog implementation: {catalog_impl}")
    print(f"   ✅ Catalog URI: {catalog_uri}")
except Exception as e:
    print(f"   ❌ Catalog not configured: {e}")

# Check 3: Network connectivity test
print("\n3️⃣ Testing network connectivity to Snowflake...")
try:
    import urllib.request
    import ssl
    
    # Test basic HTTPS connectivity to Snowflake
    context = ssl.create_default_context()
    url = f"https://{SNOWFLAKE_ACCOUNT}.snowflakecomputing.com"
    
    try:
        req = urllib.request.Request(url, method='HEAD')
        response = urllib.request.urlopen(req, context=context, timeout=10)
        print(f"   ✅ Can reach Snowflake at {url}")
        print(f"   HTTP Status: {response.status}")
    except Exception as net_err:
        print(f"   ⚠️  Network issue: {net_err}")
        
except Exception as e:
    print(f"   ⚠️  Could not test network: {e}")

# Check 4: PAT token format validation
print("\n4️⃣ Validating PAT token format...")
try:
    if ACCOUNTADMIN_PAT and len(ACCOUNTADMIN_PAT) > 50:
        print("   ✅ PAT token appears to be set (length > 50 chars)")
        # Check if it looks like a JWT
        if ACCOUNTADMIN_PAT.count('.') == 2:
            print("   ✅ PAT token format looks like a JWT (3 parts)")
        else:
            print("   ⚠️  PAT token doesn't look like a JWT")
    else:
        print("   ❌ PAT token not set or too short")
except Exception as e:
    print(f"   ❌ PAT token validation failed: {e}")

# Check 5: List all Spark catalog configurations
print("\n5️⃣ All Spark catalog configurations:")
try:
    all_configs = spark.sparkContext.getConf().getAll()
    catalog_configs = [conf for conf in all_configs if 'catalog.hz' in conf[0]]
    
    if catalog_configs:
        for key, value in catalog_configs:
            # Mask the credential
            if 'credential' in key:
                value = value[:20] + "..." if len(value) > 20 else "***"
            print(f"   {key} = {value}")
    else:
        print("   ⚠️  No catalog configurations found")
        
except Exception as e:
    print(f"   ❌ Could not list configurations: {e}")

# Check 6: Common error patterns
print("\n6️⃣ Common Issues & Solutions:")
print("   ❌ 'Cannot find catalog plugin' → Install Iceberg libraries")
print("   ❌ 'RESTException' → Check Polaris is enabled in Snowflake")
print("   ❌ '401 Unauthorized' → Verify PAT token is valid")
print("   ❌ '404 Not Found' → Check database name and Snowflake account")
print("   ❌ 'Connection refused' → Network/firewall issues")

print("\n" + "=" * 80)
print("✅ DIAGNOSTICS COMPLETE")
print("=" * 80)
